In [28]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, SimpleRNN, GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [13]:
df = pd.read_csv('../data/cleaned/campus5.csv', index_col='Timestamp')
df.index = pd.to_datetime(df.index)
df1 = pd.date_range(start = df.index[0], end = df.index[-1], freq = '15min')
df = df.reindex(df1, fill_value = 0)
df.drop(columns = ['CampusKey', 'time'], inplace = True)

In [14]:
hourly = df.resample('H').mean()
ww = df.iloc[33311:]

In [15]:
ww = ww.interpolate(method = 'linear', limit_direction = 'forward')

In [21]:
features = [i for i in ww.columns if i != 'SolarGeneration']
X = ww[features]
y = ww['SolarGeneration']
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = False, test_size = 0.2)

In [22]:
X_train

,ApparentTemperature,AirTemperature,DewPointTemperature,RelativeHumidity,WindSpeed,WindDirection
2020-12-13 06:30:00,13.500000,16.333333,9.533333,64.000000,14.20,45.133333
2020-12-13 06:45:00,13.500000,16.600000,9.800000,64.000000,15.88,43.333333
2020-12-13 07:00:00,13.673333,16.920000,9.840000,63.066667,16.74,38.666667
2020-12-13 07:15:00,13.633333,17.386667,10.040000,61.933333,19.56,40.733333
2020-12-13 07:30:00,14.386667,17.986667,10.073333,59.866667,18.90,45.266667
...,...,...,...,...,...,...
2022-01-14 09:15:00,23.150000,24.850000,17.200000,62.500000,0.00,0.000000
2022-01-14 09:30:00,23.300000,25.400000,17.100000,60.000000,0.00,0.000000
2022-01-14 09:45:00,23.600000,25.400000,17.450000,61.000000,0.00,0.000000
2022-01-14 10:00:00,23.900000,25.400000,17.800000,62.000000,0.00,0.000000


In [26]:
ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

In [29]:
train_sequences = TimeseriesGenerator(X_train_sc, y_train, length = 4, batch_size = 64)
test_sequences = TimeseriesGenerator(X_test_sc, y_test, length = 4, batch_size = 64)

In [30]:
input_shape = train_sequences[0][0][0].shape

In [33]:
model = Sequential()

model.add(GRU(8, input_shape = input_shape, return_sequences = False))
model.add(Dense(1))

model.compile(loss = 'bce', optimizer = 'adam')

res = model.fit(train_sequences, validation_data = test_sequences, epochs = 50)

Epoch 1/50
596/596 [==============================] - 6s 5ms/step - loss: 0.4156 - val_loss: -12.2547
Epoch 2/50
596/596 [==============================] - 3s 4ms/step - loss: -3.5951 - val_loss: -12.1831
Epoch 3/50
596/596 [==============================] - 2s 4ms/step - loss: -3.6962 - val_loss: -11.8237
Epoch 4/50
596/596 [==============================] - 2s 4ms/step - loss: -3.7019 - val_loss: -12.2871
Epoch 5/50
596/596 [==============================] - 2s 4ms/step - loss: -3.7146 - val_loss: -12.2070
Epoch 6/50
596/596 [==============================] - 3s 5ms/step - loss: -2.9353 - val_loss: -5.4880
Epoch 7/50
596/596 [==============================] - 3s 5ms/step - loss: -1.6561 - val_loss: -5.4445
Epoch 8/50
596/596 [==============================] - 3s 5ms/step - loss: -1.6508 - val_loss: -5.4038
Epoch 9/50
596/596 [==============================] - 3s 5ms/step - loss: -1.6424 - val_loss: -5.4052
Epoch 10/50
596/596 [==============================] - 3s 4ms/step - loss: -1.

In [2]:
np.__version__

'1.24.2'

In [3]:
!which python

/c/Users/jeffr/anaconda3/envs/myenv/python


In [4]:
!pip show numpy

Name: numpy
Version: 1.19.5
Summary: NumPy is the fundamental package for array computing with Python.
Home-page: https://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: BSD
Location: c:\users\jeffr\anaconda3\envs\myenv\lib\site-packages
Requires: 
Required-by: h5py, Keras-Preprocessing, opt-einsum, tensorboard, tensorflow


In [5]:
np.version.version

'1.24.2'

In [6]:
!pip install numpy==1.19.5